In [15]:
import os 
import numpy as np
from icecream import ic

import torch
import torch.nn as nn
import torch.nn.functional as F

In [16]:
# Path to the folder where the datasets are/should be downloaded (e.g. MNIST)
DATASET_PATH = "../data"
# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = "../saved_models/optim"

# Function for setting the seed
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
set_seed(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Fetching the device that will be used throughout this notebook
device = torch.device("cpu") if not torch.cuda.is_available() else torch.device("cuda:0")
print("Using device", device)

Using device cuda:0


In [17]:
import urllib.request
# from urllib.error import HTTPError
# Github URL where saved models are stored for this tutorial
base_url = "https://raw.githubusercontent.com/phlippe/saved_models/main/tutorial4/"
# Files to download
pretrained_files = ["FashionMNIST_SGD.config",    "FashionMNIST_SGD_results.json",    "FashionMNIST_SGD.tar",
                    "FashionMNIST_SGDMom.config", "FashionMNIST_SGDMom_results.json", "FashionMNIST_SGDMom.tar",
                    "FashionMNIST_Adam.config",   "FashionMNIST_Adam_results.json",   "FashionMNIST_Adam.tar"   ]
os.makedirs(CHECKPOINT_PATH, exist_ok=True)
for file_name in pretrained_files:
    file_path = os.path.join(CHECKPOINT_PATH, file_name) 
    if not os.path.isfile(file_path):
        file_url = base_url + file_name
        ic('downloading the file....')
        urllib.request.urlretrieve(file_url, file_path)



In [20]:
from torchvision.datasets import FashionMNIST
from torchvision import transforms
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.2861, ), (0.3530, ))])
train_dataset = FashionMNIST(root=DATASET_PATH, train=True, transform=transform)
test_set = FashionMNIST(root=DATASET_PATH, train=False, transform=transform)
train_set, val_set = torch.utils.data.random_split(train_dataset, [50000, 10000])
train_dataloader = torch.utils.data.DataLoader(dataset=train_set, batch_size=1024, shuffle=True, drop_last=False)
val_dataloader = torch.utils.data.DataLoader(dataset=val_set, batch_size=1024, shuffle=True, drop_last=False)
test_dataloader = torch.utils.data.DataLoader(dataset=test_set, batch_size=1024, shuffle=True, drop_last=False)

In [21]:
(train_dataset.data.float()/255.0).mean().item(), (train_dataset.data.float()/255.0).std().item()

(0.28604060411453247, 0.3530242443084717)